In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()

import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

(cont:sync:readmostly)=
# Read-Dominated Workloads

There are many scenarios in operating systems and other concurrent software where certain data structures are frequently read, but rarely updated. For example, the OS caches network routing information, but route updates are infrequent compared to route lookups in the cache. Since read-only accesses cannot conflict with each other, we would like to allow "readers" to execute concurrently with each other (sharing access to the data structure), while "writers" must not be allowed to conflict with either readers or other writers. We will look at two ways to solve this problem: classical reader-writer locks (rw_locks), and read-copy-update (RCU). 

(cont:sync:readmostly:rw_locks)=
## Reader-Writer Locks

Courtois et al. first described the reader-writer problem and two solutions using semaphores for mutual exclusion in 1971 {cite}`10.1145/362759.362813`. In the first version of the problem, readers have priority over writers: a reader thread should never be prevented from accessing the resource unless a writer has already been granted exclusive use of the resource. We show the solution for this reader-priority version in {numref}`listing:sync:readmostly:readerpri` using POSIX semaphores. Note that since the semaphores are used only for mutual exclusion, we could replace them with `pthread_mutex_t` or any of our spinlock implementations without making any other changes. We represent access to some shared database with the `read_db()` and `write_db()` functions. 

```{literalinclude} /src/sync/readerpri.c
:linenos:
:emphasize-lines: 11, 13, 19-25, 29-35
:start-after: Begin Courtois
:end-before: End Courtois
:name: listing:sync:readmostly:readerpri
:caption: Reader/writer solution using semaphores; readers have priority over writers.
```

In [3]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/readerpri.c", 
    lang="", 
    number=True,
    title="<b>readerpri.c - Full source code for readers/writers solution with semaphores</b>",
    h="100%", 
    w="100%"
) + '</font>'))


<font size="1.2rem"><b>readerpri.c - Full source code for readers/writers solution with semaphores</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
 1: #include <pthread.h>
 2: #include <stdlib.h>
 3: #include <sys/types.h>
 4: #include <unistd.h>
 5: #include <string.h>
 6: #include <stdio.h>
 7: #include <semaphore.h>
 8: 
 9: /* Begin Courtois et al. reader/writer semaphore solution with reader priority here */
10: extern void read_db();
11: extern void write_db();
12: 
13: sem_t counter_mutex; /* Initially 1 */
14: sem_t data_mutex;    /* Initially 1 */
15: 
16: int reader_count = 0;
17: 
18: static void *writer_thread(void *arg)
19: {
20:     sem_wait(&data_mutex);
21:     write_db();
22:     sem_post(&data_mutex);
23: }
24: 
25: static void *reader_thread(void *arg)
26: {
27:     
28:     sem_wait(&counter_mutex);
29:     reader_count++;
30:     if (reader_count == 1) {
31:         /* first reader synchronizes with writers to acquire the resource */
32:         sem_wait(&data_mutex);
33:     }
34:     sem_post(&counter_mutex);
35:     
36:     read_db();
37:     
38:     sem_wait(&counter_mutex);
39:     reader_count--;
40:     if (reader_count == 0) {
41:         /* last reader releases the resource */
42:         sem_post(&data_mutex);
43:     }
44:     sem_post(&counter_mutex);
45:     
46:     return (void *)0;
47: }
48: /* End Courtois et al. reader/writer semaphore solution with reader priority. */
49: 
50: 
51: int main(int argc, char **argv)
52: {
53:     pthread_t *tids;
54:     long niters;
55:     int nreaders, nwriters, i;
56:     
57:     if (argc != 4) {
58:         fprintf(stderr,"Usage: %s <num_readers> <num_writers> <num_iters>\n",argv[0]);
59:         return 1;
60:     }
61: 
62:     nreaders = atoi(argv[1]);
63:     nwriters = atoi(argv[2]);
64:     niters = atoi(argv[3]);
65:     
66:     printf("Main thread: Beginning test with %d readers and %d writers\n", nreaders, nwriters);
67:     sem_init(&counter_mutex, 0, 1);
68:     sem_init(&data_mutex, 0, 1);
69:     
70:     tids = (pthread_t *)malloc((nreaders + nwriters) * sizeof(pthread_t));
71:     
72:     /* We create the same number of increment and decrement threads, each doing the same number of iterations. 
73:      * When all threads have completed, we expect the final value of the shared counter to be the same as its
74:      * initial value (i.e., 0).
75:      */
76:     for (i = 0; i < nreaders; i++) {
77:         (void)pthread_create(&tids[i], NULL, reader_thread, (void *)niters );
78:     }
79:     
80:     for(i = 0; i < nwriters; i++) {
81:         (void)pthread_create(&tids[i+nreaders], NULL, writer_thread, (void *)niters );
82:     }
83:     
84:     /* Wait for child threads to finish */
85:     for (i = 0; i < (nreaders+nwriters); i++) {
86:         pthread_join(tids[i], NULL);
87:     }
88:     
89:     printf("Main thread: all done.\n");
90: 
91:     return 0;
92: }

```


</div>
</font>

The writer code (lines 11-13) is very simple. We introduce a `mutex` semaphore (line 4) to protect access to the shared data. The `wait` on `mutex` at line 11 ensures that only one writer can be accessing the data at any time; when the writer is done, it `post`s a notice to `mutex` that the data is now available again (line 13). In previous locking scenarios, the reader code would be symmetrical, with each reader also waiting for the `mutex` and `post`ing when it was done. Because we want to allow multiple readers, the reader code needs to do a little more work, however. First, we want to keep track of the number of readers currently accessing the data, so we introduce a `reader_count` variable (line 7), and since this variable will be updated by every reader thread, we also add a `counter_mutex` semaphore (line 5) to protect sections of code that depend on the `reader_count`. Each reader thread begins by `wait`ing for the `counter_mutex`, then increments `reader_count` to add itself to the count of current readers (lines 19-20). Only the first reader thread needs to synchronize with writer threads (lines 21-24)---once a reader has successfully `wait`ed for the `mutex`, writers will be locked out until all the readers are done. Notice also that the first reader does not `post` to the `counter_mutex` until *after* it has obtained permission to access the data (line 25). This ensures that other reader threads must line up `wait`ing for the `counter_mutex` as long as a writer is accessing the data. When a reader finishes using the data, it must again `wait` for the `counter_mutex` to subtract itself from the count of current readers (lines 29-30). When the `reader_count` falls to zero, it means that the last reader thread is done using the data, and this last thread must `post` a signal to the `mutex` to indicate that the data is available again (lines 31-34). If any writers are waiting, one will be allowed to proceed.

The reader code illustrates the *lightswitch pattern* in concurrent programming: the analogy is that the first person into a dark room turns on the light (acquires the resource), and the last one to leave turns it off (releases the resource). 

We can encapsulate the *entry* sections of code (line 11 for writers; lines 19-25 for readers) and the *exit* sections of code (line 13 for writers; lines 29-35 for readers) into acquire and release functions for reader/writer locks. This code is shown in {numref}`listing:sync:readmostly:rwlock`:

```{literalinclude} /src/sync/rwlock.h
:linenos:
:name: listing:sync:readmostly:rwlock
:caption: C source code for reader/writer locks using semaphores.
```


In [ ]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/readerpri.c", 
    lang="", 
    number=True,
    title="<b>rwlock.h - Source code for reader/writer locks using semaphores</b>",
    h="100%", 
    w="100%"
) + '</font>'))

We could combine the release functions and just have a single `rwlock_release()` function invoked by either reader or writer threads. When a writer releases the lock, the `reader_count` should already be `0`; in that case we skip decrementing `reader_count` and just post to the `mutex`. This is the approach taken by the pthreads library implementation of reader/writer locks (`man -k pthread_rwlock`). 

In this implementation of rwlocks, intensive read activity could starve writer threads. An alternative is to give writers priority: as soon as a writer thread requests access to the shared data, no new reader threads are allowed access until the writer is done using the data. The writer must still wait for existing reader threads to finish and release the data, but writers will not starve. If write activity is intense enough to worry about starving readers, then rwlocks are probably not a good choice anyway. 

## Performance of rwlocks

To be added. 

(cont:sync:readmostly:rcu)=
## Read-Copy Update
